# KSA206: Polar Observations and Modelling

# Week 7 - A model experiment to study future changes

<div style="float: right; margin-right: 20px; margin-left: 30px; width: 30%; max-width: 500px;">
    <img src="images/ssp_emissions.png" width="100%"/>
</div>

One of the wonderful, fun things that numerical models can be used for is to run experiments - in analogy to a lab experiment, a numerical experiment allows us to vary the model's conditions and study the impacts of such variation. Particularly, model's are widely used to study projected changes associated to different pathways of CO $_2$ emissions, the most well known of them being model's part of the Coupled Model Intercomparison Project (CMIP) that inform the Intergovernmental Panel on Climate Change (IPCC) reports. CMIP models have a coupled atmosphere, ocean and sea ice, making them quite expensive to run and producing very large outputs - which is why their resolution is not very high and there is a spectrum of processes they don't represent, particularly the introduction of freshwater into the ocean from melting ice sheets. 

A way to expand on the collective effort of CMIP is to use the atmospheric output from their projections to force a model like ACCESS-OM2 that has a higher resolution at the expense of not having an atmosphere. This week we are going to use one of those experiments that uses the CMIP6 projections for the SSP585 pathway, used in the work of [Dawson et al. 2024](). The Shared Socioeconomic Pathways (SSPs) are different possible pathways of emissions (you can see them on the figure on the right), that range from a future in which decide to actively remove anthropogenic carbon from the atmosphere (the blue lines) to futures in which we continue with "business-as-usual" emissions, no reductions (the red lines).

We will be using model simulations from the work of [Dawson et al. (2024)](). These simulations use the atmosphere from CMIP6 models at the end of the century, for the SSP585 scenario, to force ACCESS-OM2 at it's highest resolution, 0.1 $^{\circ}$. The authors also include a simulation with projected changes to freshwater input from the melting Antarctic ice sheet, obtained from [Golledge et al. (2019)](https://www.nature.com/articles/s41586-019-0889-9).

## Model simulations

In the past week we have worked with output from ACCESS-OM2 at it's lowest resolution (1 $^{\circ}$), from what we call an Interanually Forced simulation - which basically means we are forcing the model with a complete, interannually varying atmosphere from 1958 to 2018. This means we have the full spectrum of atmospheric variability in the forcing, which makes isolating the effect of a single component, like looking just at the effect of temperature or just at the effect of westerly winds, almost impossible. 

A simpler way of isolating cause and effect is to use a simulation that is run with just one year of atmospheric forcing, cycling it continuously in what is called a Repeat Year Forcing simulation. The year is chosen for being a vainilla year, with nothing especially out of the ordinary happen. We will call these our **CONTROL** simulation, and it will constitute our reference point. 

[Dawson et al. (2024)]() branched two simulation experiments from this control:

1. Wind + thermal + precipitation: changes to surface winds, temperatures and precipitation where accounted for
2. Wind + thermal + precipitation + meltwater: in addition to the above, changes to ice sheet melt and freshwater input into the ocean were included.

The process of branching is illustrated in the (very crude) schematic bellow. Note how the control (in black) and the perturbation simulations (orange and purple) are run in parallel, which allows us to compare between simulations and understand what the changes to the forcing drive:

<div align="center",style="max-width: 500px;">
    <img src="images/sim_schematic_1.png" width="60%" />
</div>

In [18]:
import cartopy.crs as ccrs
import cartopy.feature as cft
import cmocean as cm
import glob
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [19]:
def stereographic_axis(ax):
    
    ax.set_extent([-180, 180, -80, -50], crs = ccrs.PlateCarree())
    ax.add_feature(cft.NaturalEarthFeature('physical', 'land', '50m',
                                            edgecolor='gainsboro',
                                            facecolor='gainsboro'))
    ax.coastlines(resolution = "110m")
    # Map the plot boundaries to a circle
    theta = np.linspace(0, 2 * np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)

    return ax

## Perturbation forcing fields 

We will start by looking at the forcing fields used in this work. This will help us understand and interpret what we see in the simulations. 

### Load forcing field data

Let's open the atmospheric forcing fields for the control simulation, and the perturbation simulations. 

To open the files, we can use `xr.open_mfdataset` which opens and merges separate netcdf files. We need to provide the paths of the files:

In [ ]:
paths_control = glob.glob('data/forcing_fields/control*.nc') 
paths_control

In [4]:
control = xr.open_mfdataset(paths_control)

In [ ]:
control

In [ ]:
paths_perturb = glob.glob('data/forcing_fields/perturbation*.nc') 
paths_perturb

In [7]:
perturbation = xr.open_mfdataset(paths_perturb)

In [ ]:
perturbation

<h4 style="color: red;">Question 1</h4>

What variables are included in our forcing field files?

*Answer here*

Let's start by looking at zonal and meridional winds. We will plot the CONTROL forcing fields, and the anomalies applied (perturbation forcing fields *minus* control):

In [ ]:
fig = plt.figure(figsize = (15, 10))
axs = [fig.add_subplot(221, projection = ccrs.SouthPolarStereo()),
       fig.add_subplot(222, projection = ccrs.SouthPolarStereo()),
       fig.add_subplot(223, projection = ccrs.SouthPolarStereo()),
       fig.add_subplot(224, projection = ccrs.SouthPolarStereo())]
for ax in axs:
    ax = stereographic_axis(ax)

control['uas_10m'].mean('time').plot.contourf(ax = axs[0], levels = np.arange(-10, 11, 1),
                                             cmap = cm.cm.delta, transform = ccrs.PlateCarree(),
                                             cbar_kwargs = {'label':'m/s', 'shrink':0.6})
axs[0].set_title('Zonal wind, CONTROL');

(perturbation['uas_10m'] - control['uas_10m']).mean('time').plot.contourf(ax = axs[1], levels = np.arange(-2, 2.1, .1),
                                                                          cmap = 'RdBu_r', transform = ccrs.PlateCarree(),
                                                                          cbar_kwargs = {'label':'m/s', 'shrink':0.6})
axs[1].set_title('Zonal wind, PERTURBATION - CONTROL');

control['vas_10m'].mean('time').plot.contourf(ax = axs[2], levels = np.arange(-10, 11, 1),
                                             cmap = cm.cm.delta, transform = ccrs.PlateCarree(),
                                             cbar_kwargs = {'label':'m/s', 'shrink':0.6})
axs[2].set_title('Meridional wind, CONTROL');

(perturbation['vas_10m'] - control['vas_10m']).mean('time').plot.contourf(ax = axs[3], levels = np.arange(-1, 1.1, .1),
                                                                          cmap = 'RdBu_r', transform = ccrs.PlateCarree(),
                                                                          cbar_kwargs = {'label':'m/s', 'shrink':0.6})
axs[3].set_title('Meridional wind, PERTURBATION - CONTROL');

plt.tight_layout();

Note how the surface wind fields in the Southern Ocean are dominated by the zonal component - the magnitude of the zonal wind (top left) is way larger than that of the meridional wind (bottom left). 

In out perturbations, the red colors in the top right panel indicate an overall strengthening of the westerly winds, and a weakening of the near shore easterly winds (the blue, negative areas in top left). This is overall interpreted as a strengthening and shifting south of the westerly wind belt. 

The meridional perturbations are smaller in magnitude, and overall show an increase in the southward winds over most of the Southern ocean, and a small region of increased off-shore winds on the Ross Sea.

Let's look at what the changes in surface temperature look like:

In [ ]:
fig = plt.figure(figsize = (15, 10))
axs = [fig.add_subplot(121, projection = ccrs.SouthPolarStereo()),
       fig.add_subplot(122, projection = ccrs.SouthPolarStereo())]
for ax in axs:
    ax = stereographic_axis(ax)

control['tas_10m'].mean('time').plot.contourf(ax = axs[0], levels = 20,
                                             cmap = cm.cm.thermal, transform = ccrs.PlateCarree(),
                                             cbar_kwargs = {'label':'degC', 'shrink':0.4})
axs[0].set_title('Surface temperature, CONTROL');

(perturbation['tas_10m']-control['tas_10m']).mean('time').plot.contourf(ax = axs[1], levels = np.arange(-6, 6.5, .5),
                                                                        cmap = 'RdBu_r', transform = ccrs.PlateCarree(),
                                                                        cbar_kwargs = {'label':'degC', 'shrink':0.4})
axs[1].set_title('Surface temperature, PERTURBATION - CONTROL');

plt.tight_layout();

<h4 style="color: red;">Question 2</h4>

There is something wrong in our plots. Correct it.

*Answer here*

In [ ]:
control['rlds']

In [ ]:
fig = plt.figure(figsize = (15, 10))
axs = [fig.add_subplot(121, projection = ccrs.SouthPolarStereo()),
       fig.add_subplot(122, projection = ccrs.SouthPolarStereo())]
for ax in axs:
    ax = stereographic_axis(ax)

control['rlds'].mean('time').plot.contourf(ax = axs[0], levels = 20,
                                          cmap = 'gnuplot', transform = ccrs.PlateCarree(),
                                          cbar_kwargs = {'label':'W/m2', 'shrink':0.4})
axs[0].set_title('Longwave radiation, CONTROL');

(perturbation['rlds']-control['rlds']).mean('time').plot.contourf(ax = axs[1], levels = 10,
                                                                  cmap = 'Reds', transform = ccrs.PlateCarree(),
                                                                  cbar_kwargs = {'label':'W/m2', 'shrink':0.4})
axs[1].set_title('Longwave radiation, PERTURBATION - CONTROL');

plt.tight_layout();

<h4 style="color: red;">Question 3</h4>

What do you think the changes to incoming longwave radiation are due to? How are these and the surface temperature changes related?

*Answer here*

Finally, let's look at the precipitation changes. We will include liquid precipitation (rain) and solid precipitation (snow).

In [13]:
precip_control = control['prrn'] + control['prsn']
precip_perturbation = perturbation['prrn'] + perturbation['prsn']

In [ ]:
fig = plt.figure(figsize = (15, 10))
axs = [fig.add_subplot(121, projection = ccrs.SouthPolarStereo()),
       fig.add_subplot(122, projection = ccrs.SouthPolarStereo())]
for ax in axs:
    ax = stereographic_axis(ax)

precip_control.mean('time').plot.contourf(ax = axs[0], levels = 20,
                                          cmap = cm.cm.rain, transform = ccrs.PlateCarree(),
                                          cbar_kwargs = {'label':'', 'shrink':0.4})
axs[0].set_title('Precipitation, CONTROL');

(precip_perturbation-precip_control).mean('time').plot.contourf(ax = axs[1], levels = 20,
                                                                cmap = 'RdBu_r', transform = ccrs.PlateCarree(),
                                                                cbar_kwargs = {'label':'kg/m2/s', 'shrink':0.4})
axs[1].set_title('Precipitation, PERTURBATION - CONTROL');

plt.tight_layout();

<h4 style="color: red;">Question 4</h4>

Using the plots above, summarise the changes applied to the forcing fields used to run the perturbation experiments. Remember that the changes are representative of future projections for a SSP585 scenario in CMIP6 models.

*Answer here*

<h4 style="color: red;">Question 5</h4>

We have looked at the changes in the time mean forcing fields. Explore the seasonality of the changes. For example:

 - Compare changes in wind strength changes between summer and winter.
 - Surface temperatures - are they changing more in a given season?
 - Same with precipitation.

*Answer here*